# Treinos e Testes

In [ ]:
!pip install validators;
!pip install unidecode;

In [11]:
import pandas as pd
import numpy as np

# from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

import validators
from urllib.parse import urlparse

In [12]:
# Le o CSV de classificadores
classificacoes = pd.read_csv('covidbr_labeled.csv')
textos = classificacoes['text']
classes = classificacoes['misinformation']

if len(textos) != len(classes):
  print(f"Inconsistência nos dados. {len(textos)} textos e {len(classes)} classificacoes.")

In [ ]:
# Carrega stopwords do arquivo
stopwords = []
with open('stopwords.txt') as f:
  lines = f.readlines()
  stopwords = list(map(lambda x: x.strip(), lines))

# print(stopwords[:5])

In [14]:
# Tokenizacao
import unidecode

def tokeniza(xs):
  # Remove vírgulas
  nxs = [ str(texto).replace(',', ' ') for texto in xs]  

  # Remove acentos e capitalização e quebra os textos em palavras
  n2xs = [unidecode.unidecode(texto).lower().split() for texto in nxs]

  return n2xs

textos = tokeniza(textos)

if len(textos) != len(classes):
  print(f"Inconsistência nos dados. {len(textos)} textos e {len(classes)} classificacoes.")

In [ ]:
# Tratamento inicial das palavras

def trata_textos(xs):

  newTextos = []
  for lista in xs:
    if not isinstance(lista, list): continue
    newLista = []
    for palavra in lista:

      # Remove palavras vazias ou menores que 3 caracteres
      if len(palavra) <= 3: continue

      # Remove nomes de usuário
      if palavra.startswith('@'): continue

      # Remove stopwords:
      if palavra in stopwords: continue

      # Usa apenas o hostname das URLs
      if validators.url(palavra): palavra = urlparse(palavra).hostname

      # Remove pontos e espaços em branco
      palavra = palavra.replace('.', ' ').replace('?', ' ').replace(';', ' ').replace('!', ' ').replace(':', ' ').strip().split()

      for p in palavra:
        if not p.isnumeric(): newLista.append(p)

    newTextos.append(newLista)
  return newTextos

textos = trata_textos(textos)

if len(textos) != len(classes):
  print(f"Inconsistência nos dados. {len(textos)} textos e {len(classes)} classificacoes.")

In [16]:
# Cria um conjunto com todas as palavras sem repetição
palavras = { palavra for lista in textos for palavra in lista }

# Associa um ID a cada palavra
palavras = { palavra: i for i, palavra in enumerate(palavras) }

In [17]:
# Conta a presenca de cada palavra num texto
def contaPalavras(texto, palavras):
    vetor = [0] * len(palavras)
    for palavra in texto:
        if palavra in palavras:
            posicao = palavras[palavra]
            vetor[posicao] += 1
    return vetor

vetoresDeTexto = [contaPalavras(texto, palavras) for texto in textos]

In [19]:
# Armazena o texto e as classes
x = np.array(vetoresDeTexto)
y = np.array(classes)

if x.shape[0] != y.shape[0]:
  print(f"Inconsistência nos dados. {x.shape[0]} textos e {y.shape[0]} classificacoes.")

In [20]:
# Define porcentagem de treino
porcentagem_de_treino = 0.8

# Define o tamanho dos dados de treino
tamanho_de_treino = int(porcentagem_de_treino * len(y))
tamanho_de_validacao = len(y) - tamanho_de_treino

# Pega os dados de treino
treino_dados = x[:tamanho_de_treino]
treino_marcacoes = y[:tamanho_de_treino]

# Pega os dados de validacao
validacao_dados = x[tamanho_de_treino:]
validacao_marcacoes = y[tamanho_de_treino:]

In [21]:
# Treina os dados
def predict(modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k, n_jobs = -1, verbose=20)
    taxa_de_acerto = np.mean(scores)
    msg = f"Taxa de acerto: {taxa_de_acerto*100:.2f}%"
    print(msg)
    return taxa_de_acerto

In [22]:
#Utilizando o classificador OneVsRest
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
resultados = {}
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0))
# resultadoOneVsRest = predict(modeloOneVsRest, treino_dados, treino_marcacoes)
# resultados[resultadoOneVsRest] = modeloOneVsRest

In [23]:
#Utiliznado o classificador OneVsOne
from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0))
# resultadoOneVsOne = predict(modeloOneVsOne, treino_dados, treino_marcacoes)
# resultados[resultadoOneVsOne] = modeloOneVsOne

In [24]:
#Utilizando o classificador MultinomialNb
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
# resultadoMultinomial = predict(modeloMultinomial, treino_dados, treino_marcacoes)
# resultados[resultadoMultinomial] = modeloMultinomial

In [25]:
# Utilizando o classificador Adaboost
resultados = {}
modeloAdaBoost = AdaBoostClassifier()
# resultadoAdaBoost = predict(modeloAdaBoost, treino_dados, treino_marcacoes)
# resultados[resultadoAdaBoost] = modeloAdaBoost

<center>Sem tratamento:</center>

| %   | 1vs1  | 1vsR  | Multi | Ada   |
|-----|-------|-------|-------|-------|
| 0.5 | 73.70 | 73.70 | 74.59 | 71.49 |
| 0.8 | 67.01 | 67.01 | 66.49 | 69.81 |
| 1   | 65.73 | 65.73 | 65.42 | 69.68 |

<center>Tratamento completo:</center>

| %   | 1vs1  | 1vsR  | Multi | Ada   |
|-----|-------|-------|-------|-------|
| 0.5 |       |       |       |       |
| 0.8 | 84.13 | 84.13 | 84.86 | 67.96 |
| 1   | 84.17 | 84.17 | 84.67 | 81.91 |

In [26]:
modeloMultinomial.fit(x, y)
modeloOneVsOne.fit(x, y)
modeloAdaBoost.fit(x, y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


AdaBoostClassifier()

In [27]:
sc1 = modeloMultinomial.score(validacao_dados, validacao_marcacoes)
sc2 = modeloAdaBoost.score(validacao_dados, validacao_marcacoes)
sc3 = modeloOneVsOne.score(validacao_dados, validacao_marcacoes)
print(sc1, sc2, sc3)

0.8879310344827587 0.8379310344827586 0.9879310344827587


# Classificação

In [28]:
# Le o dados para classificação
conteudo = pd.read_csv('mensagens-covid.db.csv')

# Utiliza uma amostra aleatória de 10%
conteudo = conteudo.sample(frac=0.1)

mensagens = conteudo['message']
ids = list(zip(conteudo['id'],conteudo['channel_id']))

if len(mensagens) != len(ids):
  print(f"Inconsistência nos dados. {len(mensagens)} mensagens e {len(ids)} ids.")

In [29]:
# Tokeniza as mensagens
mensagens = tokeniza(mensagens)

if len(mensagens) != len(ids):
  print(f"Inconsistência nos dados. {len(mensagens)} mensagens e {len(ids)} ids.")

In [30]:
# Trata as mensagens
mensagens = trata_textos(mensagens)

if len(mensagens) != len(ids):
  print(f"Inconsistência nos dados. {len(mensagens)} mensagens e {len(ids)} ids.")

In [ ]:
print(mensagens[:5])

In [32]:
# Conta as palavras das mensagens
vetoresDeTexto = [contaPalavras(texto, palavras) for texto in mensagens]

# Armazena as mensagens e os ids
x1 = np.array(vetoresDeTexto)
y1 = np.array(ids)

if x1.shape[0] != y1.shape[0]:
  print(f"Inconsistência nos dados. {x1.shape[0]} mensagens e {y1.shape[0]} ids.")

In [33]:
p1 = modeloOneVsOne.predict(x1)
print(f"1v1: {p1}")
p2 = modeloMultinomial.predict(x1)
print(f"Multi: {p2}")
p3 = modeloAdaBoost.predict(x1)
print(f"Ada: {p3}")

1v1: [0 0 0 ... 0 0 0]
Multi: [0 1 0 ... 0 1 1]
Ada: [0 0 0 ... 0 0 0]


In [34]:
# Gerando CSVs com resultados
import csv

def gera_csv(nome,arr):
  with open(nome, 'w') as csvfile:
      fieldnames = ['id', 'channel_id', 'message', 'misinformation']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()
      for item in list(zip(y1,conteudo['message'],arr)):
        writer.writerow(
            {'id': item[0][0],
            'channel_id': item[0][1],
            'message': item[1],
            'misinformation': item[2]})
        
gera_csv('saida_1v1.csv',p1)
gera_csv('saida_multi.csv',p2)
gera_csv('saida_ada.csv',p3)

# Análise dos Resultados

In [35]:
# Lê os arquivos de resultado final
result_1v1 = pd.read_csv('saida_1v1.csv')
result_multi = pd.read_csv('saida_multi.csv')
result_ada = pd.read_csv('saida_ada.csv')

In [ ]:
def gera_csv_2(nome,df):
  with open(nome, 'w') as csvfile:
      fieldnames = ['id', 'channel_id', 'message', 'misinformation']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()
      for item in list(zip(y1,conteudo['message'],arr)):
        writer.writerow(
            {'id': item[0][0],
            'channel_id': item[0][1],
            'message': item[1],
            'misinformation': item[2]})

In [41]:
# Recupera amostras aleatórias e salva em novos CSVs

# 1v1:
sample_1v1_pos = result_1v1.loc[result_1v1['misinformation'] == 1].sample(n=20)
sample_1v1_neg = result_1v1.loc[result_1v1['misinformation'] == 0].sample(n=20)

sample_1v1 = pd.concat([sample_1v1_pos,sample_1v1_neg])
sample_1v1.to_csv('sample_1v1.csv')

# Multi:
sample_multi_pos = result_multi.loc[result_multi['misinformation'] == 1].sample(n=20)
sample_multi_neg = result_multi.loc[result_multi['misinformation'] == 0].sample(n=20)

sample_multi = pd.concat([sample_multi_pos,sample_multi_neg])
sample_multi.to_csv('sample_multi.csv')

# Ada:
sample_ada_pos = result_ada.loc[result_ada['misinformation'] == 1].sample(n=20)
sample_ada_neg = result_ada.loc[result_ada['misinformation'] == 0].sample(n=20)

sample_ada = pd.concat([sample_ada_pos,sample_ada_neg])
sample_ada.to_csv('sample_ada.csv')

## 1v1
40% de falsos positivos, 20% de falsos negativos

## Multi
10% de falsos positivos, 20% de falsos negativos

## Ada
40% de falsos positivos, 20% de falsos negativos
